# Setup

In [ ]:
!pip install torch torchvision
!pip install git+https://github.com/modAL-python/modAL.git
!pip install skorch

In [ ]:

#@title import libraries
import random
from google.colab import drive
import torch
import torch.nn as nn
import numpy as np
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import json
import os
import pdb
from modAL.models import ActiveLearner
from modAL.uncertainty import uncertainty_sampling
from modAL.uncertainty import margin_sampling
from modAL.uncertainty import entropy_sampling
from skorch import NeuralNetClassifier
from torch.utils.data import DataLoader, TensorDataset
from torchvision.transforms import ToTensor
from sklearn.metrics import accuracy_score
from skorch.callbacks import Callback
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import precision_recall_fscore_support
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

# utilities

In [ ]:
# Define the categories using a dictionary
category_dict = {"X": "block",
                 "/": "structure1",
                 "\\": "structure2",
                 "#": "Enemy",
                 "*": "switch",
                 "^": "spike",
                 "&": "food",
                 "-": "food"}

def merge_dict_keys(dict_in):
    # create a new dictionary with merged keys
    dict_out = {}
    for key, value in dict_in.items():
        if value not in dict_out:
            dict_out[key] = value
        else:
            dict_out[key] += ', ' + value

    # remove redundant key-value pairs
    dict_out = {v: k for k, v in dict_out.items()}
    return {v: k for k, v in dict_out.items()}

merged_category_dict = merge_dict_keys(category_dict)
print(merged_category_dict)
def create_matrix(string):
    # Split the string into rows using newline characters.
    rows = string.split('\n')
    # Remove any empty rows.
    rows = [row for row in rows if row]
    # Split each row into columns using individual characters.
    matrix = []
    for row in rows:
        columns = [char for char in row]
        matrix.append(columns)

    # Return the resulting matrix.
    return matrix
def replace_characters(matrix):
    # Define the characters to replace and their replacements.
    replacements = {'\\': '/', '&':'-'}
    #  Create a new matrix to store the modified elements.
    new_matrix = []
    for row in matrix:
        new_row = []
        for element in row:
            # If the current element is in the replacements dictionary, replace it.
            if element in replacements:
                new_row.append(replacements[element])
            else:
                new_row.append(element)
        new_matrix.append(new_row)

    # Return the modified matrix.
    return new_matrix
def create_binary_matrices(matrix):
    # Define the characters to search for in the matrix.
    characters = {'X', '/', '#', '*', '^', '-'}

    # Determine the dimensions of the matrix.
    num_rows = len(matrix)
    num_cols = len(matrix[0])

    # Create a dictionary to store the binary matrices.
    binary_matrices = {}

    # Loop through each character and create a binary matrix for that character.
    for character in characters:
        # Create a NumPy array to store the binary matrix.
        binary_matrix = np.zeros((num_rows, num_cols))

        # Loop through each element in the matrix and set the corresponding
        # element in the binary matrix to 1 if it matches the current character.
        for i in range(num_rows):
            for j in range(num_cols):
                if matrix[i][j] == character:
                    binary_matrix[i][j] = 1

        # Add the binary matrix to the dictionary.
        binary_matrices[character] = binary_matrix

    # Return the dictionary of binary matrices.
    return binary_matrices

{'X': 'block', '/': 'structure1', '\\': 'structure2', '#': 'Enemy', '*': 'switch', '^': 'spike', '-': 'food'}


# mount files

In [ ]:
#@title mounting files

# extracting labels
drive.mount('/content/drive')
gram_dict = {}
map_dict = {}
ngram_dict = {}

# gram elite info
with open('/content/drive/MyDrive/DungeonData/gram_elites/generate_corpus_info.json') as f:
    data = json.load(f)
    for filename in data['fitness'].keys():
      gram_dict[filename] = data['fitness'][filename]
print("gram_elite_info.json imported")

# map elite info
with open('/content/drive/MyDrive/DungeonData/map_elites/generate_corpus_info.json') as f:
    data = json.load(f)
    for filename in data['fitness'].keys():
      map_dict[filename] = data['fitness'][filename]
print("map_elites_info.json imported")

# n gram info
with open('/content/drive/MyDrive/DungeonData/n_gram/generate_corpus_info.json') as f:
    data = json.load(f)
    for filename in data['fitness'].keys():
      ngram_dict[filename] = data['fitness'][filename]
print("ngram_info.json imported")

# extracting levels
drive.mount('/content/drive')

playable_level_strings = []
unplayable_level_strings = []
playable_level_labels = []
unplayable_level_labels = []

#gram elite
for filename in os.listdir('/content/drive/MyDrive/DungeonData/gram_elites/levels'):
  if filename.endswith('.txt'):
      with open(os.path.join('/content/drive/MyDrive/DungeonData/gram_elites/levels', filename), 'r') as file:
          file_contents = file.read()
          if filename in gram_dict:
            if gram_dict[filename] != 0:
              playable_level_strings.append(file_contents)
              playable_level_labels.append(1)
            else:
              unplayable_level_strings.append(file_contents)
              unplayable_level_labels.append(0)
print("gram elite levels imported")

#map elite
for filename in os.listdir('/content/drive/MyDrive/DungeonData/map_elites/levels'):
  if filename.endswith('.txt'):
      with open(os.path.join('/content/drive/MyDrive/DungeonData/map_elites/levels', filename), 'r') as file:
          file_contents = file.read()
          if filename in map_dict:
            if map_dict[filename] != 0:
              playable_level_strings.append(file_contents)
              playable_level_labels.append(1)
            else:
              unplayable_level_strings.append(file_contents)
              unplayable_level_labels.append(0)
print("map elite levels imported")

#n gram
for filename in os.listdir('/content/drive/MyDrive/DungeonData/n_gram/levels'):
  if filename.endswith('.txt'):
      with open(os.path.join('/content/drive/MyDrive/DungeonData/n_gram/levels', filename), 'r') as file:
          file_contents = file.read()
          if filename in ngram_dict:
            if ngram_dict[filename] != 0:
              playable_level_strings.append(file_contents)
              playable_level_labels.append(1)
            else:
              unplayable_level_strings.append(file_contents)
              unplayable_level_labels.append(0)
print("n gram levels imported")

In [ ]:
#@title balance the dataset
from imblearn.over_sampling import ADASYN
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler


#print("number of playable levels before balancing : %4d| number of unplayable levels before balancing : %4d|" % (len(playable_level_strings), len(unplayable_level_strings)))
# Reshape X to a 2D array
#X_flat = np.reshape(levels, (levels.shape[0], -1))
# Reshape y to a 2D array
#y_flat = np.reshape(labels, (-1, 1))
#oversampler = RandomUnderSampler()
#X_resampled_flat, y_resampled_flat = oversampler.fit_resample(X_flat, y_flat)
# Reshape the 2D array back to the original shape
#X_resampled = np.reshape(X_resampled_flat, (-1, levels.shape[1], levels.shape[2], levels.shape[3]))
# Reshape the y 2D array back to the original shape
#y_resampled = np.reshape(y_resampled_flat, (-1,))

#print(X_resampled.shape)
#print(y_resampled.shape)
#print("number of playable levels after balancing : %4d| number of unplayable levels after balancing : %4d|" % (np.count_nonzero(y_resampled == 0), np.count_nonzero(y_resampled == 1)))
# balance the dataset
print("number of playable levels before balancing : %4d| number of unplayable levels before balancing : %4d|" % (len(playable_level_strings), len(unplayable_level_strings)))
n = len(unplayable_level_strings) - len(playable_level_strings)
for i in range(n):
    index = random.randint(0, len(unplayable_level_strings)-1)
    unplayable_level_strings.pop(index)
    unplayable_level_labels.pop(index)
print("number of playable levels after balancing : %4d| number of unplayable levels after balancing : %4d|" % (len(playable_level_strings), len(unplayable_level_strings)))


In [ ]:
#@title convert level strings to one-hot matrices

#@title convert level strings to one-hot matrices
print(len(playable_level_strings))
print(len(unplayable_level_strings))
level_strings = []
level_strings.extend(playable_level_strings)
level_strings.extend(unplayable_level_strings)
labels = []
labels.extend(playable_level_labels)
labels.extend(unplayable_level_labels)
labels = np.array(labels)

playable_idx = np.random.choice(range(len(playable_level_strings)), size=5, replace=False)
unplayable_idx = np.random.choice(range(len(unplayable_level_strings)), size=5, replace=False)
initial_x = []
initial_y = []

for i in playable_idx:
  matrix = create_matrix(playable_level_strings[i])
  replaced = replace_characters(matrix)
  binary = create_binary_matrices(replaced)
  initial_x.append(np.array(list(binary.values())))
  initial_y.append(1)

for i in playable_idx:
  matrix = create_matrix(playable_level_strings[i])
  replaced = replace_characters(matrix)
  binary = create_binary_matrices(replaced)
  initial_x.append(np.array(list(binary.values())))
  initial_y.append(0)

# Define the size of the level
# Split the level into individual rows
rows = level_strings[0].strip().split('\n')
# Find the width and height of the level
width = len(rows)
height = len(rows[0])
num_levels = len(level_strings)
levels = np.zeros((num_levels, 6,width,height))

for i, level_string in enumerate(level_strings):
    matrix = create_matrix(level_string)
    replaced = replace_characters(matrix)
    binary = create_binary_matrices(replaced)
    levels[i] = np.array(list(binary.values()))
print('the overal shape of X dataset: ' + str(levels.shape))
print('the overal shape of Y dataset: ' + str(labels.shape))

# Model

In [ ]:

class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()

        self.conv1 = nn.Conv2d(6, 16, kernel_size=3, padding=1)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(kernel_size=2)

        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(kernel_size=2)

        self.conv3 = nn.Conv2d(32, 64, kernel_size=2, padding=0)
        self.relu3 = nn.ReLU()

        self.fc1 = nn.Linear(128, 32)
        self.relu4 = nn.ReLU()

        self.fc2 = nn.Linear(32, 2)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.pool1(x)

        x = self.conv2(x)
        x = self.relu2(x)
        x = self.pool2(x)

        x = self.conv3(x)
        x = self.relu3(x)
        x = torch.flatten(x, 1)

        x = self.fc1(x)
        x = self.relu4(x)

        x = self.fc2(x)
        x = self.sigmoid(x)

        return x

# Passive Learner

In [ ]:
from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix
import time

def encode_labels(labels):
    if not isinstance(labels, torch.Tensor) or labels.dtype != torch.float32:
        raise ValueError("Input must be a PyTorch tensor of float32 dtype.")
    if labels.ndim != 1 or not torch.all(torch.logical_or(labels == 0, labels == 1)):
        raise ValueError("Input must be a 1D PyTorch tensor of 0s and 1s.")
    encoded_labels = torch.zeros((len(labels), 2), dtype=torch.float32)
    encoded_labels[torch.where(labels == 0)[0], 0] = 1
    encoded_labels[torch.where(labels == 1)[0], 1] = 1
    return encoded_labels

X_tensor = torch.FloatTensor(levels)
y_tensor = torch.FloatTensor(labels)
device = "cuda" if torch.cuda.is_available() else "cpu"
dataset = torch.utils.data.TensorDataset(torch.tensor(X_tensor).to(device), torch.tensor(y_tensor).to(device))
# Assign 20% of the data to the validation set
train_data, test_data = train_test_split(dataset, test_size=0.2, random_state=42)

#uncomment if want
# Assign 20% of the data to the validation set
#trainval_data, test_data = train_test_split(dataset, test_size=0.2, random_state=42)
# Assign 600 random samples of the dataset to the training set
#train_data, val_data = torch.utils.data.random_split(trainval_data, [600, len(trainval_data) - 600])

print(len(test_data))
print(len(train_data))

# Define the number of folds for cross-validation on the validation set
num_folds = 5

# Use scikit-learn's KFold to split the validation set into folds
kf = KFold(n_splits=num_folds, shuffle=True)


# Initialize lists to accumulate the true and predicted labels for all folds
all_true_labels = []
all_pred_labels = []
all_fold_acc_history = []
start_time = time.time()

# Iterate over the folds on the validation set
for fold, (train_indices, val_indices) in enumerate(kf.split(test_data)):

    # Create a new instance of your model for each fold
    model = Model()
    criterion = nn.BCELoss()
    optimizer = torch.optim.Adam(model.parameters(), weight_decay=0.01)

    # Create DataLoader objects for the training and validation sets for this fold
    train_fold_data = torch.utils.data.Subset(train_data, train_indices)
    val_fold_data = torch.utils.data.Subset(test_data, val_indices)
    train_fold_loader = torch.utils.data.DataLoader(train_fold_data, batch_size=len(train_fold_data), shuffle=True)
    val_fold_loader = torch.utils.data.DataLoader(val_fold_data, batch_size=len(val_fold_data), shuffle=True)

    fold_acc_history = []
    # Train the model for this fold
    for epoch in range(100):
        train_loss = 0
        train_acc = 0
        for inputs, labels in train_fold_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, encode_labels(labels))
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
        train_loss /= len(val_fold_loader)
        # Print the training loss and accuracy for this epoch
        print(f"Validation fold {fold + 1}, epoch {epoch + 1}: train_loss = {train_loss:.2f}")

# Evaluate the model on the validation set for this fold
        with torch.no_grad():
            model.eval()
            val_loss = 0
            val_acc = 0
            true_labels = []
            pred_labels = []
            for inputs, labels in val_fold_loader:
                outputs = model(inputs)
                loss = criterion(outputs, encode_labels(labels))
                val_loss += loss.item()
                y_pred = np.argmax(outputs, axis=1)
                true_labels += labels.tolist()
                pred_labels += y_pred.round().tolist()
            val_loss /= len(val_fold_loader)

        # Print the validation loss and accuracy for this fold
        val_acc = accuracy_score(true_labels, pred_labels)
        fold_acc_history.append(val_acc)
        # Print the validation loss and accuracy for this fold
        print(f"Fold {fold + 1}: val_loss = {val_loss:.2f}, val_acc = {val_acc:.2f}")
        # Add the accuracy history for this fold to the list of all fold accuracy histories
    if(np.mean(fold_acc_history) > 0.5) :
        all_fold_acc_history.append(fold_acc_history)
        # Accumulate the true and predicted labels for all folds
        all_true_labels += true_labels
        all_pred_labels += pred_labels

# Compute the overall confusion matrix and accuracy
overall_cm = confusion_matrix(all_true_labels, all_pred_labels)
overall_acc = accuracy_score(all_true_labels, all_pred_labels)

# Print the overall confusion matrix and accuracy
print(f"Overall confusion matrix:\n{overall_cm}")
print(f"Overall accuracy: {overall_acc:.2f}")

# Compute the average accuracy over all folds for each epoch
mean_acc_history = [sum([fold_acc_history[i] for fold_acc_history in all_fold_acc_history])/num_folds for i in range(len(all_fold_acc_history[0]))]

# Plot the average accuracy over all folds over time
plt.plot(mean_acc_history)
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Average Accuracy over Time')
plt.show()
end_time = time.time()
execution_time = end_time - start_time
print(f"Execution time of: {execution_time} seconds")

import pickle

with open('/content/drive/MyDrive/Zelda-data/passive-accuracy-400-last.pickle', 'wb') as handle:
    pickle.dump(all_fold_acc_history, handle)

with open('/content/drive/MyDrive/Zelda-data/passive-confusion-400-last.pickle', 'wb') as handle:
    pickle.dump(overall_cm, handle)

with open('/content/drive/MyDrive/Zelda-data/passive-execution-400-last.pickle', 'wb') as handle:
    pickle.dump(execution_time, handle)


# Active Learner

In [ ]:
def random_query_strategy(classifier, X, n_instances=1):
    # Generate a list of indices to select random instances from X
    indices = list(range(len(X)))
    # Shuffle the indices to select random instances
    random.shuffle(indices)
    # Return the first n_instances instances from the shuffled indices
    return indices[:n_instances]

#random_query_strategy
#margin_sampling
#entropy_sampling
#uncertainty_sampling

In [ ]:
#@title active learner
from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix
import pdb
import math
import time

#random_query_strategy
#margin_sampling
#entropy_sampling
#uncertainty_sampling

def train_active_learner(strategy, n_queries, n_instances, max_epochs):
    # Use scikit-learn's KFold to split the validation set into folds
    num_folds = 5
    kf = KFold(n_splits=num_folds, shuffle=True)
    # Initialize lists to accumulate the true and predicted labels for all folds
    all_fold_acc_history = []
    avg_false_positives = []
    avg_false_negatives = []
    all_c = []
    # Create DataLoader objects for the training and validation sets for this fold
    X_tensor = torch.FloatTensor(levels)
    y_tensor = torch.LongTensor(labels)
    device = "cuda" if torch.cuda.is_available() else "cpu"
    dataset = torch.utils.data.TensorDataset(torch.tensor(X_tensor).to(device), torch.tensor(y_tensor).to(device))
    dataset_loader = torch.utils.data.DataLoader(dataset, batch_size=len(dataset))
    X , y = next(iter(dataset_loader))
    X = X.detach().cpu().numpy()
    y = y.detach().cpu().numpy()

    X_ini = torch.FloatTensor(initial_x)
    y_ini = torch.LongTensor(initial_y)
    ini_dataset = torch.utils.data.TensorDataset(torch.tensor(X_ini).to(device), torch.tensor(y_ini).to(device))
    ini_loader = torch.utils.data.DataLoader(ini_dataset, batch_size=len(ini_dataset))
    X_0 , y_0 = next(iter(ini_loader))
    X_0 = X_0.detach().cpu().numpy()
    y_0 = y_0.detach().cpu().numpy()

    # initialize ActiveLearner
    #n_initial = 10
    #initial_idx = np.random.choice(range(len(dataset)), size=n_initial, replace=False)
    #data_initial = torch.utils.data.Subset(dataset, initial_idx)
    #data_initial_loader = torch.utils.data.DataLoader(data_initial, batch_size=len(data_initial))
    #X_0 , y_0 = next(iter(data_initial_loader))
    #X_0 = X_0.detach().cpu().numpy()
    #y_0 = y_0.detach().cpu().numpy()

    # Iterate over the folds on the validation set
    start_time = time.time()
    for fold, (train_indices, val_indices) in enumerate(kf.split(dataset)):
        X_train = X[train_indices]
        X_test= X[val_indices]
        y_train = y[train_indices]
        y_test = y[val_indices]
        false_negatives = []
        false_positives = []

        print(f"Fold no {fold+1}")
        # Create a new instance of your model for each fold
        classifier = NeuralNetClassifier(Model,
                                 criterion=nn.CrossEntropyLoss,
                                 optimizer=torch.optim.Adam,
                                 optimizer__weight_decay=0.01,
                                 max_epochs = max_epochs,
                                 train_split=None,
                                 verbose=0,
                                 device=device,
                                 warm_start = True,
                                 )
        learner = ActiveLearner(estimator=classifier,X_training=X_0, y_training=y_0,query_strategy=strategy)
        y_pred = learner.predict(X_test)
        ini_acc = accuracy_score(y_test, y_pred)
        print(ini_acc)
        fold_acc_history = []
        #trained_x = np.empty((0,8,14,25))
        #trained_y = np.empty((0,))

        for idx in range(n_queries):
              false_positives = 0
              false_negatives = 0
              fold_fn = []
              fold_fp = []
            #for k in range(0,5):
              random_indices = np.random.choice(range(len(X_train)), size=math.floor(0.5*len(X_train)), replace=False)

              query_idx, query_instance = learner.query(X_train[random_indices], n_instances=n_instances)
              #print("query length ", len(query_idx))
              '''
              if idx == 0:
                trained_x = x_query
                trained_y = y_query
              else:
                trained_x = np.append(trained_x,x_query, axis=0)
                trained_y = np.append(trained_y,y_query, axis = 0)
              #print("trained length: ", len(trained_x))
              learner.teach(X=trained_x, y=trained_y)
              '''
              x_query = X_train[query_idx]
              y_query = y_train[query_idx]
              learner.teach(X=x_query, y=y_query)
              # Evaluate the performance of the active learning model
              y_pred = learner.predict(X_test)
              true_labels = y_test.tolist()
              pred_labels = y_pred.tolist()
              # Compute FP and FN

              #false_positives = np.count_nonzero(np.logical_and((pred_labels == 1),(true_labels == 0)))
              #false_negatives = np.count_nonzero(np.logical_and((pred_labels == 0),(true_labels == 1)))
              #pdb.set_trace()
              val_acc = accuracy_score(y_test, y_pred)
              fold_acc_history.append(val_acc)
              # remove queried instance from pool
              X_train = np.delete(X_train, query_idx, axis=0)
              y_train = np.delete(y_train, query_idx, axis=0)
              print(f"Query no {idx+1}: Acc = {val_acc:.2f}:")
              #fold_fn.append(false_negatives)
              #fold_fp.append(false_positives)
              #c = confusion_matrix(true_labels, pred_labels)

        for kk in range(len(true_labels)):
                if (pred_labels[kk] == 1) & (true_labels[kk] == 0):
                  false_positives += 1
                if (pred_labels[kk] == 0) & (true_labels[kk] == 1):
                  false_negatives += 1
        c = confusion_matrix(true_labels, pred_labels)
        #if np.mean(np.array(fold_acc_history))>0.51:
        all_fold_acc_history.append(fold_acc_history)
          # Compute the average FP and FN for all folds
        #avg_false_positives.append(fold_fp)
        #avg_false_negatives.append(fold_fn)
        all_c.append(c)

    # Compute the overall confusion matrix and accuracy
    #overall_cm = confusion_matrix(all_true_labels, all_pred_labels)
    #overall_acc = accuracy_score(all_true_labels, all_pred_labels)

    # Compute the average accuracy over all folds for each epoch
    #mean_acc_history = [sum([fold_acc_history[i] for fold_acc_history in all_fold_acc_history])/num_folds for i in range(len(all_fold_acc_history[0]))]
    #mean_acc_history = [ini_acc] + mean_acc_history
    end_time = time.time()
    execution_time = end_time - start_time

    return  all_fold_acc_history , execution_time , all_c



In [ ]:
#@title margin sampling
import pickle

STRATEGY = margin_sampling
N_INSTANCES = 1
N_QUERIES = 400
MAX_EPOCHS = 50
np.random.seed(60)

# Define the size of the level
width = 29
height = 13
num_levels = len(level_strings)
levels = np.zeros((num_levels, 6,11,15))

for i, level_string in enumerate(level_strings):
    matrix = create_matrix(level_string)
    replaced = replace_characters(matrix)
    binary = create_binary_matrices(replaced)
    levels[i] = np.array(list(binary.values()))
labels = []
labels.extend(playable_level_labels)
labels.extend(unplayable_level_labels)
labels = np.array(labels)
print('the overal shape of X dataset: ' + str(levels.shape))
print('the overal shape of Y dataset: ' + str(labels.shape))

all_fold_acc_history_1, execution_time_1, overall_cm_1= train_active_learner(STRATEGY, N_QUERIES, N_INSTANCES,MAX_EPOCHS)

print(f"Execution time: {execution_time_1} seconds")
print(f"confusion matrix: {overall_cm_1}")

# Extract false positives and false negatives from each matrix
fps = [np.mean(cm[0][1]) for cm in overall_cm_1]
fns = [np.mean(cm[1][0]) for cm in overall_cm_1]

print(fps)
print(fns)

# Calculate average accuracy and standard error for each epoch
mean_accuracy_1 = np.mean(all_fold_acc_history_1, axis=0)
std_error_1 = np.std(all_fold_acc_history_1, axis=0) / np.sqrt(len(all_fold_acc_history_1))
std_dev_1 = np.std(all_fold_acc_history_1, axis=0)
# Plotting
epochs_1 = range(1, len(mean_accuracy_1) + 1)
# Calculate upper and lower bounds for the fill region
lower_bound_1 = mean_accuracy_1 - std_dev_1
upper_bound_1 = mean_accuracy_1 + std_dev_1

plt.errorbar(epochs_1, mean_accuracy_1, yerr=std_error_1, capsize=3)
plt.xlabel('Queries')
plt.ylabel('Accuracy')
plt.title('Average Accuracy over Epoch with Standard Error')
plt.show()

plt.plot(epochs_1, mean_accuracy_1, label='Average Accuracy')
plt.fill_between(epochs_1, lower_bound_1, upper_bound_1, alpha=0.3, label='Standard Deviation')
plt.xlabel('Queries')
plt.ylabel('Accuracy')
plt.title('Average Accuracy over Queries with Standard Deviation')
plt.legend()
plt.show()

with open('/content/drive/MyDrive/Zelda-data/margin-accuracy-400-last.pickle', 'wb') as handle:
    pickle.dump(all_fold_acc_history_1, handle)

with open('/content/drive/MyDrive/Zelda-data/margin-confusion-400-last.pickle', 'wb') as handle:
    pickle.dump(overall_cm_1, handle)

with open('/content/drive/MyDrive/Zelda-data/margin-execution-400-last.pickle', 'wb') as handle:
    pickle.dump(execution_time_1, handle)

In [ ]:
#@title random sampling
STRATEGY = random_query_strategy
N_INSTANCES = 1
N_QUERIES = 400
MAX_EPOCHS = 50
np.random.seed(50)

# Define the size of the level
width = 29
height = 13
num_levels = len(level_strings)
levels = np.zeros((num_levels, 6,11,15))

for i, level_string in enumerate(level_strings):
    matrix = create_matrix(level_string)
    replaced = replace_characters(matrix)
    binary = create_binary_matrices(replaced)
    levels[i] = np.array(list(binary.values()))
labels = []
labels.extend(playable_level_labels)
labels.extend(unplayable_level_labels)
labels = np.array(labels)
print('the overal shape of X dataset: ' + str(levels.shape))
print('the overal shape of Y dataset: ' + str(labels.shape))

all_fold_acc_history_2, execution_time_2, overall_cm_2 = train_active_learner(STRATEGY, N_QUERIES, N_INSTANCES,MAX_EPOCHS)
print(f"Execution time: {execution_time_2} seconds")
print(f"confusion matrix: {overall_cm_2}")
# Extract false positives and false negatives from each matrix
fps = [np.mean(cm[0][1]) for cm in overall_cm_1]
fns = [np.mean(cm[1][0]) for cm in overall_cm_1]

print(fps)
print(fns)
# Calculate average accuracy and standard error for each epoch
mean_accuracy_2 = np.mean(all_fold_acc_history_2, axis=0)
std_error_2 = np.std(all_fold_acc_history_2, axis=0) / np.sqrt(len(all_fold_acc_history_2))
std_dev_2 = np.std(all_fold_acc_history_2, axis=0)
# Plotting
epochs_2 = range(1, len(mean_accuracy_2) + 1)
# Calculate upper and lower bounds for the fill region
lower_bound_2 = mean_accuracy_2 - std_dev_2
upper_bound_2 = mean_accuracy_2 + std_dev_2

plt.errorbar(epochs_2, mean_accuracy_2, yerr=std_error_2, capsize=3)
plt.xlabel('Queries')
plt.ylabel('Accuracy')
plt.title('Average Accuracy over Epoch with Standard Error')
plt.show()

plt.plot(epochs_2, mean_accuracy_2, label='Average Accuracy')
plt.fill_between(epochs_2, lower_bound_2, upper_bound_2, alpha=0.3, label='Standard Deviation')
plt.xlabel('Queries')
plt.ylabel('Accuracy')
plt.title('Average Accuracy over Queries with Standard Deviation')
plt.legend()
plt.show()


with open('/content/drive/MyDrive/Zelda-data/random-accuracy-400-last.pickle', 'wb') as handle:
    pickle.dump(all_fold_acc_history_2, handle)

with open('/content/drive/MyDrive/Zelda-data/random-confusion-400-last.pickle', 'wb') as handle:
    pickle.dump(overall_cm_2, handle)

with open('/content/drive/MyDrive/Zelda-data/random-execution-400-last.pickle', 'wb') as handle:
    pickle.dump(execution_time_2, handle)



# plots

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

print(len(all_fold_acc_history_2))
epochs_1 = range(11, len(mean_accuracy_1) + 11)
epochs_2 = range(11, len(mean_accuracy_1) + 11)

plt.plot(epochs_1, mean_accuracy_1, color='b', linestyle='-', label='Uncertainty Sampling')
plt.fill_between(epochs_1, lower_bound_1, upper_bound_1, alpha=0.3)
plt.plot(epochs_2, mean_accuracy_2,color='r', linestyle='-', label='Random Sampling')
plt.fill_between(epochs_2, lower_bound_2, upper_bound_2, alpha=0.3)
plt.axhline(y=0.81, color='g', linestyle='--', label="Passive Learner (1206 samples)")
plt.ylim(0.4, 1)
plt.xlim(10, 410)
plt.xlabel('#Samples')
plt.ylabel('Accuracy')
plt.title('Average Accuracy over Samples')
plt.legend()
plt.show()



plt.plot(epochs_1, mean_accuracy_1, color='b', linestyle='-', label='Uncertainty Sampling')
plt.fill_between(epochs_1, lower_bound_1, upper_bound_1, alpha=0.3)
plt.plot(epochs_2, mean_accuracy_2,color='r', linestyle='-', label='Random Sampling')
plt.fill_between(epochs_2, lower_bound_2, upper_bound_2, alpha=0.3)
plt.axhline(y=0.81, color='g', linestyle='--', label="Passive Learner (1206 samples)")
plt.xlabel('#Samples')
plt.ylabel('Accuracy')
#plt.xticks(np.arange(10,410, 50))
plt.title('Average Accuracy over Samples')
plt.legend()
plt.show()
